# 📊 Project: Data Quality Framework for BSE Market Instruments

This project automates the download, loading, and validation of BSE BhavCopy data. It applies key data quality (DQ) checks to ensure consistency, completeness, and accuracy of market data.

## 📁 1. Data Download and Setup

Import necessary Libriaries

In [1]:
import os
import requests
from datetime import datetime, timedelta
import time
import pandas as pd
import re

Code for downloading BhavCopy CSV files from BSE's public archive.

In [2]:
def clear_folder(folder_path):
    """Delete all files in the specified folder."""
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print(f"🧹 Cleared folder: {folder_path}")
    else:
        os.makedirs(folder_path)
        print(f"📂 Created folder: {folder_path}")

def download_bse_bhavcopy(start_date, end_date, download_path="bse_bhavcopies"):
    base_url = "https://www.bseindia.com/download/BhavCopy/Equity/BhavCopy_BSE_CM_0_0_0_{date}_F_0000.CSV"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.bseindia.com/',
        'Connection': 'keep-alive'
    }

    clear_folder(download_path)

    current = start_date
    while current <= end_date:
        if current.weekday() < 5:  # Skip weekends
            date_str = current.strftime("%Y%m%d")
            file_url = base_url.format(date=date_str)
            filename = f"BhavCopy_BSE_{date_str}.csv"
            file_path = os.path.join(download_path, filename)

            try:
                response = requests.get(file_url, headers=headers, timeout=10)
                content_type = response.headers.get("Content-Type", "")

                if response.status_code == 200:
                    with open(file_path, "wb") as f:
                        f.write(response.content)
                    print(f"✅ Downloaded: {filename}")
                else:
                    print(f"❌ Not found or wrong type: {filename} | Content-Type: {content_type}")

            except Exception as e:
                print(f"⚠️ Error downloading {filename}: {e}")

            time.sleep(1.5)  # Be polite to the server
        current += timedelta(days=1)
# Download Files
start = datetime(2025, 3, 1)
end = datetime(2025, 3, 31)
download_bse_bhavcopy(start, end)


🧹 Cleared folder: bse_bhavcopies
✅ Downloaded: BhavCopy_BSE_20250303.csv
✅ Downloaded: BhavCopy_BSE_20250304.csv
✅ Downloaded: BhavCopy_BSE_20250305.csv
✅ Downloaded: BhavCopy_BSE_20250306.csv
✅ Downloaded: BhavCopy_BSE_20250307.csv
✅ Downloaded: BhavCopy_BSE_20250310.csv
✅ Downloaded: BhavCopy_BSE_20250311.csv
✅ Downloaded: BhavCopy_BSE_20250312.csv
✅ Downloaded: BhavCopy_BSE_20250313.csv
❌ Not found or wrong type: BhavCopy_BSE_20250314.csv | Content-Type: text/html; charset=utf-8
✅ Downloaded: BhavCopy_BSE_20250317.csv
✅ Downloaded: BhavCopy_BSE_20250318.csv
✅ Downloaded: BhavCopy_BSE_20250319.csv
✅ Downloaded: BhavCopy_BSE_20250320.csv
✅ Downloaded: BhavCopy_BSE_20250321.csv
✅ Downloaded: BhavCopy_BSE_20250324.csv
✅ Downloaded: BhavCopy_BSE_20250325.csv
✅ Downloaded: BhavCopy_BSE_20250326.csv
✅ Downloaded: BhavCopy_BSE_20250327.csv
✅ Downloaded: BhavCopy_BSE_20250328.csv
❌ Not found or wrong type: BhavCopy_BSE_20250331.csv | Content-Type: text/html; charset=utf-8


## 📥 2. Data Loading and Cleaning

Load all downloaded files, inspect structure, and prepare for analysis.

In [3]:
Data = []

for filename in os.listdir('bse_bhavcopies'):
    file_path = os.path.join('bse_bhavcopies', filename)
    df = pd.read_csv(file_path)
    date_str = filename.replace("BhavCopy_BSE_", "").replace(".csv", "")
    df['Date'] = pd.to_datetime(date_str, format="%Y%m%d")
    Data.append(df) 
combined_df = pd.concat(Data, ignore_index=True)
combined_df.columns = combined_df.columns.str.strip() 

In [4]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86134 entries, 0 to 86133
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   TradDt               86134 non-null  object        
 1   BizDt                86134 non-null  object        
 2   Sgmt                 86134 non-null  object        
 3   Src                  86134 non-null  object        
 4   FinInstrmTp          86134 non-null  object        
 5   FinInstrmId          86134 non-null  int64         
 6   ISIN                 86134 non-null  object        
 7   TckrSymb             86134 non-null  object        
 8   SctySrs              86134 non-null  object        
 9   XpryDt               0 non-null      float64       
 10  FininstrmActlXpryDt  0 non-null      float64       
 11  StrkPric             0 non-null      float64       
 12  OptnTp               0 non-null      float64       
 13  FinInstrmNm          86134 non-

In [5]:
combined_df.head(20)

,TradDt,BizDt,Sgmt,Src,FinInstrmTp,FinInstrmId,ISIN,TckrSymb,SctySrs,XpryDt,...,TtlTrfVal,TtlNbOfTxsExctd,SsnId,NewBrdLotQty,Rmks,Rsvd1,Rsvd2,Rsvd3,Rsvd4,Date
0,2025-03-03,2025-03-03,CM,BSE,STK,500002,INE117A01022,ABB,A,NaN,...,59771184.0,2658,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
1,2025-03-03,2025-03-03,CM,BSE,STK,500003,INE208C01025,AEGISLOG,A,NaN,...,174431287.0,4944,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
2,2025-03-03,2025-03-03,CM,BSE,STK,500008,INE885A01032,ARE&M,A,NaN,...,142627397.0,11951,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
3,2025-03-03,2025-03-03,CM,BSE,STK,500009,INE432A01017,AMBALALSA,X,NaN,...,7002903.0,918,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
4,2025-03-03,2025-03-03,CM,BSE,STK,500012,INE714B01016,ANDHRAPET,X,NaN,...,4915213.0,793,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
5,2025-03-03,2025-03-03,CM,BSE,STK,500013,INE436A01026,ANSALAPI,Z,NaN,...,285522.0,54,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
6,2025-03-03,2025-03-03,CM,BSE,STK,500014,INE096A01010,UTIQUE,X,NaN,...,152274.0,187,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
7,2025-03-03,2025-03-03,CM,BSE,STK,500016,INE957C01019,ARUNAHTEL,X,NaN,...,188345.0,52,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
8,2025-03-03,2025-03-03,CM,BSE,STK,500020,INE032A01023,BOMDYEING,A,NaN,...,15322985.0,1677,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03
9,2025-03-03,2025-03-03,CM,BSE,STK,500023,INE363A01022,ASIANHOTNR,T,NaN,...,35644.0,1,F1,1,NaN,NaN,NaN,NaN,NaN,2025-03-03


In [6]:
# bhav_format_df
pd.set_option('display.max_columns', None)

bhav_format_df = pd.read_excel('UDiFF_BhavCopy_format.xlsx')
bhav_format_df.columns = bhav_format_df.columns.str.strip() 
bhav_format_df

,Sr. No.,Field Name,Data Type,ISO Tags,CM,FO,CD,COM,Data Format ( if any),Field Description
0,1,Trade Date,Varchar (10)\nFormat (YYYY-MM-DD),TradDt,Yes,Yes,Yes,Yes,Format (YYYY-MM-DD),Trade Date
1,2,Business Date,Varchar (10)\nFormat (YYYY-MM-DD),BizDt,Yes,Yes,Yes,Yes,Format (YYYY-MM-DD),This will be date when the session has started
2,3,Segment Indicator,Varchar (5),Sgmt,Yes,Yes,Yes,Yes,-,"Refer sheet ""Standard Value List""\n\nCM=Cash M..."
3,4,Source,Varchar (10),Src,Yes,Yes,Yes,Yes,-,"Refer sheet ""Standard Value List"""
4,5,Instrument Type,Varchar (5),FinInstrmTp,Yes,Yes,Yes,Yes,-,"\n\nCM= STK\n\nFO = STF (Futures Stock),STO (O..."
5,6,Unique Instrument Identifier Of Trading Exchange,NUMERIC(10),FinInstrmId,Yes,Yes,Yes,Yes,-,Scrip Code/Contract Token Number
6,7,Instrument ISIN,VarChar (12),ISIN,Yes,Blank,Blank,Blank,-,ISIN
7,8,Instrument Symbol or Scrip Code,Varchar(12),TckrSymb,Yes,Yes,Yes,Yes,-,Symbol
8,9,Instrument Series,Varchar(4),SctySrs,Yes,Blank,Blank,Blank,-,Scrip Group under which the scrip is traded
9,10,Instrument Original Expiry Date,Varchar (10)\nFormat (YYYY-MM-DD),XpryDt,Blank,Yes,Yes,Yes,Format (YYYY-MM-DD),Instrument Original Expiry Date is the expiry ...


In [7]:
# Rename columns
bhav_format_df['Field Name'] = bhav_format_df['Field Name'].str.strip().str.replace(' ', '_').str.lower()

new_columns = {}
for index, row in bhav_format_df.iterrows():
    new_columns[row['ISO Tags']] = row['Field Name']
combined_df = combined_df.rename(columns = new_columns)

#Filter Columns
keep_columns = ['business_date', 
               'instrument_isin', 
               'instrument_symbol_or_scrip_code',
               'instrument_series', 
               'open', 
               'high', 
               'low', 
               'close',
               'ltp', 
               'previous_close', 
               'settlement_price', 
               'total_traded_qty',
               'total_traded_value', 
               'total_number_of_trades']
combined_df = combined_df[keep_columns]
bhav_format_df = bhav_format_df[bhav_format_df['Field Name'].isin(keep_columns)]

In [8]:
# Column description
bhav_format_df = bhav_format_df.copy()
bhav_format_df['Field Description'] = bhav_format_df['Field Description'].astype(str).str.split('\n').str[0]
description = bhav_format_df[['Field Name', 'Field Description']]
combined_df.attrs['column_descriptions'] = description.to_dict()['Field Description']

for col, desc in zip(description['Field Name'], description['Field Description']):
    print(f"{col}: {desc}")



business_date: This will be date when the session has started
instrument_isin: ISIN
instrument_symbol_or_scrip_code: Symbol
instrument_series: Scrip Group under which the scrip is traded
open: Opening Price
high: High Price for the Day
low: Low Price for the Day
close: Closing Price for the Day
ltp: Last Traded Price
previous_close: Previous Day's Closing Price
settlement_price: Max values populated as 19 numeric digits before decimal and 6 numeric digits after decimal.
total_traded_qty: Total Traded Qty
total_traded_value: Max values populated as 19 numeric digits before decimal and 6 numeric digits after decimal.
total_number_of_trades: Total Number of Trades


## 📊 3. Data Quality Rules

Define and implement checks like null values, duplicates, outliers, and anomalies.

In [9]:
# Initialize summary list
dq_summary = []

# 1. Null Check
null_check_fields = [
    'instrument_isin', 'instrument_symbol_or_scrip_code', 'close', 'total_traded_qty'
]
for field in null_check_fields:
    total = len(combined_df)
    null_issues = combined_df[field].isnull().sum()
    dq_summary.append({
        "DQ Rule": f"Null Check: {field}",
        "Records Checked": total,
        "Issues Found": null_issues,
        "Pass %": round(100 * (1 - null_issues / total), 2)
    })
print("🔍 Null Value Issues:\n", null_issues)

🔍 Null Value Issues:
 0


In [10]:
# 2. Duplicate Check
total = len(combined_df)
duplicates = combined_df.duplicated(
    subset=['business_date', 'instrument_symbol_or_scrip_code']
).sum()
dq_summary.append({
    "DQ Rule": "Duplicate Check: trade_date + symbol",
    "Records Checked": total,
    "Issues Found": duplicates,
    "Pass %": round(100 * (1 - duplicates / total), 2)
})
print(f"🔍 Duplicate Records Found: {duplicates}")

🔍 Duplicate Records Found: 0


In [11]:
# 3. Price Outlier Check

combined_df['close'] = pd.to_numeric(combined_df['close'], errors='coerce')
combined_df['previous_close'] = pd.to_numeric(combined_df['previous_close'], errors='coerce')
combined_df['price_change_pct'] = ((combined_df['close'] - combined_df['previous_close']) / combined_df['previous_close']) * 100
outliers = combined_df[combined_df['price_change_pct'].abs() > 100]
dq_summary.append({
    "DQ Rule": "Price Change > 100%",
    "Records Checked": total,
    "Issues Found": len(outliers),
    "Pass %": round(100 * (1 - len(outliers) / total), 2)
})
print(f"⚠️ Outliers in Close Price (>100% change): {len(outliers)}")

⚠️ Outliers in Close Price (>100% change): 105


These are likely data quality issues (e.g. bad imports, split adjustments not accounted for, or real but extreme volatility).

In [12]:
# 4. Volume Anomaly (>10000 median)
volume_anomaly = combined_df[
    combined_df['total_traded_qty'] > 10000 * combined_df['total_traded_qty'].median()
]
dq_summary.append({
    "DQ Rule": "Volume > 10000 Median",
    "Records Checked": total,
    "Issues Found": len(volume_anomaly),
    "Pass %": round(100 * (1 - len(volume_anomaly) / total), 2)})
print(f"📈 Volume Anomalies (>10000 median): {len(volume_anomaly)}")

📈 Volume Anomalies (>10000 median): 6


In [13]:
# 5. ISIN Format Validation
isin_pattern = re.compile(r"^IN[A-Z0-9]{10}$")
invalid_isins = combined_df[~combined_df['instrument_isin'].astype(str).str.match(isin_pattern)]

dq_summary.append({
    "DQ Rule": "ISIN Format Validation",
    "Records Checked": total,
    "Issues Found": len(invalid_isins),
    "Pass %": round(100 * (1 - len(invalid_isins) / total), 2)})

print(f"🔍 Invalid ISIN Format: {len(invalid_isins)} records")

🔍 Invalid ISIN Format: 0 records


In [14]:
# 6. Price Field Consistency
price_errors = combined_df[
    (combined_df['low'] > combined_df['close']) |
    (combined_df['close'] > combined_df['high']) |
    ((combined_df[['low', 'close', 'high']] < 0).any(axis=1))]

dq_summary.append({
    "DQ Rule": "Price Erroe",
    "Records Checked": total,
    "Issues Found": len(price_errors),
    "Pass %": round(100 * (1 - len(price_errors) / total), 2)})

print(f"📉 Price Consistency Errors: {len(price_errors)} records")

📉 Price Consistency Errors: 2 records


In [15]:
# 7. Trade Value
trade_qv_errors = combined_df[
    (combined_df['total_traded_qty'] > 0) &
    (combined_df['total_traded_value'] <= 0)]

dq_summary.append({
    "DQ Rule": "Total Traded Value <= 0",
    "Records Checked": total,
    "Issues Found": len(trade_qv_errors),
    "Pass %": round(100 * (1 - len(trade_qv_errors) / total), 2)})

print(f"🔢 Invalid Quantity/Value: {len(trade_qv_errors)} records")

🔢 Invalid Quantity/Value: 2 records


## 4. Summary of Issues

### ✅ Key Findings:

In [16]:
# Convert to DataFrame for display
dq_summary_df = pd.DataFrame(dq_summary)
print(dq_summary_df.to_string(index=False))

                                    DQ Rule  Records Checked  Issues Found  Pass %
                Null Check: instrument_isin            86134             0  100.00
Null Check: instrument_symbol_or_scrip_code            86134             0  100.00
                          Null Check: close            86134             0  100.00
               Null Check: total_traded_qty            86134             0  100.00
       Duplicate Check: trade_date + symbol            86134             0  100.00
                        Price Change > 100%            86134           105   99.88
                      Volume > 10000 Median            86134             6   99.99
                     ISIN Format Validation            86134             0  100.00
                                Price Erroe            86134             2  100.00
                    Total Traded Value <= 0            86134             2  100.00
